In [1]:
import jax
import jax.numpy as jnp
import sys
sys.path.append('..')

import analysis
import datatypes
from input_pipeline_tf import get_datasets

2023-03-07 18:51:06.210312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 18:51:07.582071: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 18:51:07.582147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 18:51:07.582155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:
config, best_state, metrics_for_best_state = analysis.load_from_workdir('../workdirs/mace0')

cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

qm9_datasets = get_datasets(key, config)
example_graph = next(qm9_datasets["test"].as_numpy_iterator())
frag = datatypes.Fragment.from_graphstuple(example_graph)
frag = jax.tree_map(jnp.asarray, frag)

/home/songk/atomicarchitects/spherical_harmonic_net/qm9_data_tf/
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-07 18:51:12.100850: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 18:51:12.101151: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-07 18:51:12.101304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-07 18:51:12.101444: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-03-07 18:51:12.101587: W tensorfl

In [ ]:
# generated_dict = analysis.to_mol_dict(frag, "../workdirs", "generated_single_frag")

0 1
1 3
3 6
6 10
10 15
15 16
16 18
18 21
21 25
25 26
26 28
28 31
31 32
32 34
34 37
37 41
41 42
42 44
44 47
47 48
48 50
50 53
53 57
57 58
58 60
60 63
63 67
67 72
72 78
78 85
85 93
93 94
94 96
96 99
99 103
103 108
108 114
114 115
115 117
117 120
120 124
124 129
129 135
135 142
142 143
143 145
145 148
148 152
152 157
157 163
163 164
164 166
166 169
169 173
173 178
178 184
184 191
191 192
192 194
194 197
197 201
201 206
206 212
212 512


{1: [{'_positions': array([[-0.5238,  1.4379,  0.9064]], dtype=float32),
   '_atomic_numbers': array([1])},
  {'_positions': array([[ 0.9158,  1.3587, -0.0288]], dtype=float32),
   '_atomic_numbers': array([1])},
  {'_positions': array([[-0.0344,  0.9775,  0.0076]], dtype=float32),
   '_atomic_numbers': array([8])},
  {'_positions': array([[-1.6616,  0.    ,  1.    ]], dtype=float32),
   '_atomic_numbers': array([1])},
  {'_positions': array([[-0.0133,  1.1325,  0.0083]], dtype=float32),
   '_atomic_numbers': array([6])},
  {'_positions': array([[0.915 , 1.7895, 0.004 ]], dtype=float32),
   '_atomic_numbers': array([1])},
  {'_positions': array([[0.9949, 1.9397, 0.0029]], dtype=float32),
   '_atomic_numbers': array([1])},
  {'_positions': array([[-0.0083,  1.3905, -0.0056]], dtype=float32),
   '_atomic_numbers': array([6])},
  {'_positions': array([[0.9978, 1.8743, 0.0026]], dtype=float32),
   '_atomic_numbers': array([1])},
  {'_positions': array([[-0.0179,  1.4671,  0.0101]], dtype=f